In [6]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
df_ = pd.read_excel("online_retail_II.xlsx",
                    sheet_name="Year 2010-2011")

In [4]:
df = df_.copy()

In [7]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

In [8]:
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

In [9]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541910.00000,9.55223,218.08096,-80995.00000,1.00000,3.00000,10.00000,80995.00000
Price,541910.00000,4.61114,96.75977,-11062.06000,1.25000,2.08000,4.13000,38970.00000
Customer ID,406830.00000,15287.68416,1713.60307,12346.00000,13953.00000,15152.00000,16791.00000,18287.00000


In [10]:
df.dropna(inplace=True)
df = df[~df["Invoice"].str.contains("C", na=False)]
df = df[df["Quantity"] > 0]

In [11]:
replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "Price")
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,397925.00000,11.83371,25.53449,1.00000,2.00000,6.00000,12.00000,298.50000
Price,397925.00000,2.89320,3.22714,0.00000,1.25000,1.95000,3.75000,37.06000
Customer ID,397925.00000,15294.30860,1713.17274,12346.00000,13969.00000,15159.00000,16795.00000,18287.00000


In [12]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [13]:
today_date = dt.datetime(2011, 12, 11)

In [14]:
rfm = df.groupby('Customer ID').agg({'InvoiceDate': [lambda date: (today_date - date.max()).days,
                                                     lambda date: (today_date - date.min()).days],
                                     'Invoice': lambda num: num.nunique(),
                                     'TotalPrice': lambda price: price.sum()})

In [15]:
rfm.columns = rfm.columns.droplevel(0)
rfm.columns = ['Recency', "T", 'Frequency', "Monetary"]
rfm.head()

,Recency,T,Frequency,Monetary
Customer ID,,,,
12346.00000,326,326,1,310.44000
12347.00000,3,368,7,4310.00000
12348.00000,76,359,4,1770.78000
12349.00000,19,19,1,1491.72000
12350.00000,311,311,1,331.46000


In [17]:
temp_df = df.groupby(["Customer ID", "Invoice"]).agg({"Invoice": "count", "TotalPrice": ["mean"]})
temp_df.head()

Invoice TotalPrice
                      count       mean
Customer ID Invoice                   
12346.00000 541431        1  310.44000
12347.00000 537626       31   22.96097
            542237       29   16.39276
            549222       24   26.51042
            556201       18   21.25111

In [18]:
temp_df = temp_df.reset_index()
temp_df.columns = temp_df.columns.droplevel(0)
temp_df.columns = ["Customer ID", "Invoice", "total_price_count", "total_price_mean"]

In [19]:
temp_df2 = temp_df.groupby(["Customer ID"], as_index=False).agg({"total_price_mean": ["mean"]})
temp_df2.head()

,Customer ID,total_price_mean
,,mean
0,12346.00000,310.44000
1,12347.00000,23.09700
2,12348.00000,66.01550
3,12349.00000,20.43452
4,12350.00000,19.49765


In [20]:
temp_df2.columns = temp_df2.columns.droplevel(0)
temp_df2.columns = ["Customer ID", "monetary_avg"]

In [21]:
rfm.index.isin(temp_df2["Customer ID"]).all()

True

In [22]:
rfm = rfm.merge(temp_df2, how="left", on="Customer ID")
rfm.set_index("Customer ID", inplace=True)
rfm.head()

,Recency,T,Frequency,Monetary,monetary_avg
Customer ID,,,,,
12346.00000,326,326,1,310.44000,310.44000
12347.00000,3,368,7,4310.00000,23.09700
12348.00000,76,359,4,1770.78000,66.01550
12349.00000,19,19,1,1491.72000,20.43452
12350.00000,311,311,1,331.46000,19.49765


In [26]:
rfm.index = rfm.index.astype(int)
rfm["Recency_weekly"] = rfm["Recency"] / 7
rfm["T_weekly"] = rfm["T"] / 7
rfm = rfm[(rfm['monetary_avg'] > 0)]
rfm_cltv["Frequency"] = rfm_cltv["Frequency"].astype(int)
rfm_cltv = rfm.copy()
rfm_cltv.head()

,Recency,T,Frequency,Monetary,monetary_avg,Recency_weekly,T_weekly
Customer ID,,,,,,,
12346,326,326,1,310.44000,310.44000,46.57143,46.57143
12347,3,368,7,4310.00000,23.09700,0.42857,52.57143
12348,76,359,4,1770.78000,66.01550,10.85714,51.28571
12349,19,19,1,1491.72000,20.43452,2.71429,2.71429
12350,311,311,1,331.46000,19.49765,44.42857,44.42857


In [27]:
# BG/NBD 
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(rfm_cltv['Frequency'],
        rfm_cltv['Recency_weekly'],
        rfm_cltv['T_weekly'])

<lifetimes.BetaGeoFitter: fitted with 4338 subjects, a: 1.52, alpha: 0.07, b: 5.69, r: 0.28>

In [34]:
# Top 10 customer who will make the most purchases in 1 month
bgf.conditional_expected_number_of_purchases_up_to_time(4,
                                                        rfm_cltv['Frequency'],
                                                        rfm_cltv['Recency_weekly'],
                                                        rfm_cltv['T_weekly']).sort_values(ascending=False).head(10)

Customer ID
16000   7.13233
12713   5.10115
15060   4.84868
18139   4.81851
13298   4.14386
15520   4.14386
14569   4.14386
13436   4.14386
15195   3.52802
15471   3.52802
dtype: float64

In [35]:
rfm_cltv["expected_number_of_purchases"] = bgf.predict(4,
                                                       rfm_cltv['Frequency'],
                                                       rfm_cltv['Recency_weekly'],
                                                       rfm_cltv['T_weekly'])

rfm_cltv.head()

,Recency,T,Frequency,Monetary,monetary_avg,Recency_weekly,T_weekly,expected_number_of_purchases
Customer ID,,,,,,,,
12346,326,326,1,310.44000,310.44000,46.57143,46.57143,0.08517
12347,3,368,7,4310.00000,23.09700,0.42857,52.57143,0.00000
12348,76,359,4,1770.78000,66.01550,10.85714,51.28571,0.00244
12349,19,19,1,1491.72000,20.43452,2.71429,2.71429,1.13720
12350,311,311,1,331.46000,19.49765,44.42857,44.42857,0.08919


In [36]:
# Total expected selling number in 1 month
bgf.predict(4,
            rfm_cltv['Frequency'],
            rfm_cltv['Recency_weekly'],
            rfm_cltv['T_weekly']).sum()

883.6007950590689

In [37]:
# Gamma Gamma
ggf = GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(rfm_cltv['Frequency'], rfm_cltv['monetary_avg'])

ggf.conditional_expected_average_profit(rfm_cltv['Frequency'],
                                        rfm_cltv['monetary_avg']).head(10)

Customer ID
12346   313.89978
12347    23.56336
12348    66.83891
12349    23.69915
12350    22.76164
12352    23.05688
12353    25.51585
12354    21.87374
12355    38.61312
12356    39.49249
dtype: float64

In [38]:
ggf.conditional_expected_average_profit(rfm_cltv['Frequency'],
                                        rfm_cltv['monetary_avg']).sort_values(ascending=False).head(10)

Customer ID
16000   1188.52359
16532   1123.69478
15749    970.89486
15098    853.82762
15195    824.67825
18102    634.92178
13270    593.64789
18080    568.96565
17291    554.42152
16698    530.14009
dtype: float64

In [39]:
rfm_cltv["expected_average_profit"] = ggf.conditional_expected_average_profit(rfm_cltv['Frequency'],
                                                                              rfm_cltv['monetary_avg'])

In [44]:
# BG-NBD & GAMMA GAMMA -> CLTV
cltv = ggf.customer_lifetime_value(bgf,
                                   rfm_cltv['Frequency'],
                                   rfm_cltv['Recency_weekly'],
                                   rfm_cltv['T_weekly'],
                                   rfm_cltv['monetary_avg'],
                                   time=3,  # 3 MONTHS
                                   freq="W",  # T's frequence info.
                                   discount_rate=0.01)
cltv.head()

Customer ID
12346   82.21368
12347    0.00000
12348    0.49106
12349   60.95285
12350    6.23263
Name: clv, dtype: float64

In [45]:
cltv = cltv.reset_index()

In [46]:
rfm_cltv_final = rfm_cltv.merge(cltv, on="Customer ID", how="left")
rfm_cltv_final.head()

,Customer ID,Recency,T,Frequency,Monetary,monetary_avg,Recency_weekly,T_weekly,expected_number_of_purchases,expected_average_profit,clv
0,12346,326,326,1,310.44000,310.44000,46.57143,46.57143,0.08517,313.89978,82.21368
1,12347,3,368,7,4310.00000,23.09700,0.42857,52.57143,0.00000,23.56336,0.00000
2,12348,76,359,4,1770.78000,66.01550,10.85714,51.28571,0.00244,66.83891,0.49106
3,12349,19,19,1,1491.72000,20.43452,2.71429,2.71429,1.13720,23.69915,60.95285
4,12350,311,311,1,331.46000,19.49765,44.42857,44.42857,0.08919,22.76164,6.23263
